In [1]:
import numpy as np
import pandas as pd

data = pd.read_csv("CPI_changed.csv")
data = data.rename(columns={'1 Consumer Price Index  (2012=100)': 'CPI', 'Item': 'Date'})
data = data.dropna()
data['Date'] = pd.to_datetime(data['Date'], format='%b-%y')
data = data.set_index('Date').asfreq('MS')
data['CPI'] = pd.to_numeric(data['CPI'].str.replace('-', ''))
df = pd.DataFrame(data['CPI'])
df['inflation'] = df['CPI'].pct_change(fill_method=None)
df = df.dropna()
df

,CPI,inflation
Date,,
2013-02-01,105.3,0.006692
2013-03-01,105.5,0.001899
2013-04-01,106.1,0.005687
2013-05-01,106.9,0.007540
2013-06-01,109.3,0.022451
...,...,...
2025-08-01,197.0,0.004589
2025-09-01,197.0,0.000000
2025-10-01,197.3,0.001523


In [ ]:
mask = (df.index >= '2015-01-01') & (df.index <= '2024-12-31')
train = df.loc[mask]
temask = (df.index >= '2025-01-01') & (df.index <= '2025-12-31')
test = df.loc[temask]

In [3]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

alpha = float(input("Enter Alpha: (0-1)") or 0.6)
beta = float(input("Enter Beta: (0-1)") or 0.3)
gamma = float(input("Enter Gamma: (0-1)") or 0.38)


In [ ]:
fit3 = ExponentialSmoothing(train['CPI'], trend='mul', seasonal='mul', 
                            seasonal_periods=12).fit(smoothing_level=alpha, smoothing_trend=beta, smoothing_seasonal= gamma, optimized=False)

pred3 = fit3.forecast(len(test))

# mask = (df.index >= '2020-01-01') & (df.index <= '2024-12-31')
# train_df = df.loc[mask]
last_train_date = train.index[-1]
last_train_val = train['CPI'].iloc[-1]

dates = np.insert(test.index, 0, last_train_date)
predict_values = np.insert(pred3, 0, last_train_val)
test_values = np.insert(test['CPI'], 0, last_train_val)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=train.index, y=train['CPI'], name='Train'))
fig.add_trace(go.Scatter(x=dates, y=test_values, name='Test'))
fig.add_trace(go.Scatter(x=dates, y=predict_values, name='Holt Winters Prediction'))
fig.update_layout(template="simple_white", font=dict(size=18), title_text="CPI: Actual + Predicted",
                  width=1000, title_x=0.5, height=600,
                  yaxis_title='CPI')
fig.show()

In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
mean = test.mean()['CPI']
mae = mean_absolute_error(test['CPI'], pred3)
mse = mean_squared_error(test['CPI'], pred3)
rmse = np.sqrt(mse)

mae_pct = (mae/mean)*100
rmse_pct = (rmse/mean)*100

print(f"MAE: {mae_pct:.2f}%")
print(f"RMSE: {rmse_pct:.2f}%")

MAE: 0.47%
RMSE: 0.53%


In [ ]:
fit = ExponentialSmoothing(df['CPI'], trend='mul', seasonal='mul', 
                            seasonal_periods=12).fit(smoothing_level=alpha, smoothing_trend=beta, smoothing_seasonal=gamma, optimized=False)

mask = (df.index >= '2023-01-01') & (df.index <= '2025-12-31')
df2 = df.loc[mask]

pred_final = fit.forecast(4)
date_list = pd.date_range(start = '2026-01-01', periods = 4, freq = 'MS')

last_train_date = df2.index[-1]
last_train_val = df2['CPI'].iloc[-1]

pred_dates = np.insert(date_list, 0, last_train_date)
pred_values = np.insert(pred_final, 0, last_train_val)

fig = go.Figure()
fig.add_trace(go.Scatter(x=df2.index, y=df2['CPI'], name='Train'))
fig.add_trace(go.Scatter(x=pred_dates, y=pred_values, name='Holt Winters Forecast'))
fig.update_layout(template="simple_white", font=dict(size=18), title_text="CPI: Actual + Forecasted",
                  width=1000, title_x=0.5, height=600,
                  yaxis_title='CPI')
fig.show()